In [1]:
import torch
import numpy as np
import os
import pandas as pd
from torch.utils.data import Dataset

In [2]:

# dataset_path = os.path.join("..","Datasets","TRACEPARTS_DATA")

# dataset_path

In [3]:
# json_path_train = os.path.join(dataset_path,"train_test_split/clean_shuffled_train_file_list.json")
# json_path_val = os.path.join(dataset_path,"train_test_split/clean_shuffled_val_file_list.json")

In [4]:
# json_train = pd.read_json(json_path_train)
# json_val = pd.read_json(json_path_val)

In [5]:
# json_train = json_train.replace(to_replace=r"^shape_data",value=dataset_path,inplace=False,regex=True) + ".txt" 
# json_val = json_val.replace(to_replace=r"^shape_data",value=dataset_path,inplace=False,regex=True)+".txt"
# json_train.to_csv(os.path.join(dataset_path,"train.csv"),index=None)
# json_val.to_csv(os.path.join(dataset_path,"validation.csv"),index=None)
# train = pd.read_csv(os.path.join(dataset_path,"train.csv"))
# list(train["0"])

In [6]:
class TraceParts_Dataloader(Dataset):
    def __init__(self,sample_points=1024, split='train', normalize=True, cache_size=150,uniform=True,normals=True):
        self.sample_points = sample_points
        self.split = split
        self.normalize = normalize
        self.cache_size = cache_size
        self.uniform = uniform
        self.normals = normals
        self.dataset_path = os.path.join("..","Datasets","TRACEPARTS_DATA")
        train_path = os.path.join(self.dataset_path,"train.csv")
        val_path = os.path.join(self.dataset_path,"validation.csv")
        self.train_files = list(pd.read_csv(train_path)["0"])
        self.val_files = list(pd.read_csv(val_path)["0"])
        self.split_dic = {"train":self.train_files,"val":self.val_files}
        self.cache = {}
    
    
    def pc_normalize(self,pc):
        centroid = np.mean(pc, axis=0)
        pc = pc - centroid
        m = np.max(np.sqrt(np.sum(pc**2, axis=1)))
        pc = pc / m
        return pc
    
    def farthest_point_sample(self,point,label,npoint):
        """
        Input:
            xyz: pointcloud data, [N, D]
            label : per point label ,[N]
            npoint: number of samples
        Return:
            centroids: sampled pointcloud index, [npoint, D]
        """
        N, D = point.shape
        xyz = point[:,:3]
        centroids = np.zeros((npoint,))
        distance = np.ones((N,)) * 1e10
        farthest = np.random.randint(0, N)
        for i in range(npoint):
            centroids[i] = farthest
            centroid = xyz[farthest, :]
            dist = np.sum((xyz - centroid) ** 2, -1)
            mask = dist < distance
            distance[mask] = dist[mask]
            farthest = np.argmax(distance, -1)
        point = point[centroids.astype(np.int32)]
        label = label[centroids.astype(np.int32)]
        return point,label
        
    def __len__(self):
        return len(self.split_dic[self.split])
    
    def __getitem__(self,index):
        if index in self.cache:
            return self.cache[index][0],self.cache[index][1]
        else:
            file = np.loadtxt(self.split_dic[self.split][index])
            if self.normals:
                pc,label = file[0:-1],file[:,-1]
            else:
                pc,label = file[0:3],file[:,-1]
            
            if self.uniform:
                pc,label = self.farthest_point_sample(pc,label,self.sample_points)
            else:
                pc,label = pc[0:self.sample_points,:],label[0:self.sample_points]
            
            if len(self.cache) < self.cache_size:
                self.cache[index] = (pc, label)
            
            return pc,label
                
                
                
            
        
        
        

In [7]:
x = torch.randn((16,1024,3))
dis = torch.cdist(x,x)
dis.shape
ne = dis.topk(dim=2,k=32,largest=False)
ne.indices.shape

torch.Size([16, 1024, 32])

In [101]:
def save_as_ply(name,pc,center,idx):
    #pred = np.asarray(pred.cpu())
    # pred = np.ones((pc.shape[0],1))*3
    #pc = np.asarray(pc.cpu())
    # pred = pred.reshape(pred.shape[0],1)
    file = open(name+".ply",'w')
    n_verts = pc.shape[0]
    file.write("ply\nformat ascii 1.0\nelement vertex "+str(n_verts)+"\nproperty float x\nproperty float y\nproperty float z\nproperty uchar red\nproperty uchar green\nproperty uchar blue\nend_header\n")
    color_map = {0:"0 0 0", 1:"0 0 255", 2:"255 0 255", 3:"0 255 0"}
    # encode_pred = ''
    file.write(str(center[0])+" "+str(center[1])+" "+str(center[2])+" "+"0 255 0\n")
#     print(idx)
    for i in range(pc.shape[0]):
        ek = str(pc[i][0])
        do = str(pc[i][1])
        ten = str(pc[i][2])
        #char = str(pc[i][3])
        #pan = str(pc[i][4])
        #she = str(pc[i][5])
        if pc[i] in idx:
            rgb = color_map[2]
        else:
            rgb = "0 0 0"
        #file.write(ek+" "+do+" "+ten+" "+char+" "+pan+" "+she+" "+rgb+'\n')
        file.write(ek+" "+do+" "+ten+" "+rgb+'\n')

    file.close()

In [108]:
i = 5
j = 100
save_as_ply("test",pc[i,:,:3].numpy(),pc[i,j,:3].numpy(),c[i,j,:,:3].numpy())

In [8]:
train_data = TraceParts_Dataloader(split="val")
train_loader = torch.utils.data.DataLoader(train_data,batch_size=16,shuffle=True)
cuda = torch.device('cuda')
import time
for batch,(pc,label) in enumerate(train_loader):
    print(batch)
#     pc.cuda()
    s_t = time.time()
    dis = torch.cdist(pc[:,:,:3],pc[:,:,:3])
    neighbors = dis.topk(dim=2,k=32,largest=False)
#     pc_with_neighbors = torch.gather(pc,1,neighbors.indices)
#     print(pc_with_neighbors.shape)
    print("Time Taken :",time.time()-s_t)
#     print(neighbors.indices[0][0][:].numpy())
    save_as_ply(str(batch)+"_0",pc[0,:,:3].numpy(),pc[0][0][:3].numpy(),neighbors.indices[0][0][:].numpy())
    save_as_ply(str(batch)+"_1",pc[1,:,:3].numpy(),pc[1][0][:3].numpy(),neighbors.indices[1][0][:].numpy())
    save_as_ply(str(batch)+"_2",pc[2,:,:3].numpy(),pc[2][0][:3].numpy(),neighbors.indices[2][0][:].numpy())
    
    
    
    

0
Time Taken : 0.2815415859222412


NameError: name 'save_as_ply' is not defined

In [9]:
pc.shape

torch.Size([16, 1024, 7])

In [10]:
def knn(pc,n_neighbors=32):
    """
    Input:
        pc : Point cloud (B,N,d) knn is computed in the d dimension eculidean spcae with the euclidean distance
        n_neighbors : Number of neighbors to find default is 32
    Return:
        indices of the n_neighbors for each point (B,N,n_neighbors)
    """

    dist = torch.cdist(pc,pc)
    neigbhors = dist.topk(k=n_neighbors,dim=2,largest=False)
    return neigbhors.indices

In [14]:
def index_points(points, idx):
    """
    Input:
        points: input points data, [B, N, C]
        idx: sample index data, [B, S]
    Return:
        new_points:, indexed points data, [B, S, C]
    """
    device = points.device
    B = points.shape[0]
    view_shape = list(idx.shape)
    print(view_shape)
    view_shape[1:] = [1] * (len(view_shape) - 1)
    print(view_shape)
    repeat_shape = list(idx.shape)
    repeat_shape[0] = 1
    print(repeat_shape)
    batch_indices = torch.arange(B, dtype=torch.long).to(device).view(view_shape).repeat(repeat_shape)
    print(batch_indices[0].shape)
    new_points = points[batch_indices, idx, :]
    return new_points

In [15]:
knn_idx = knn(pc[:,:,:3])

In [16]:
knn_pc = index_points(pc,knn_idx)
knn_pc.shape

[16, 1024, 32]
[16, 1, 1]
[1, 1024, 32]
torch.Size([1024, 32])


torch.Size([16, 1024, 32, 7])

In [30]:
def group_norm(points,group_points):
    B,S,K,D = points.size()
    group_points_norm = group_points - points.view(B,S,1,D)
    return group_points_norm

In [29]:
grouped = group_norm(pc,knn_pc)
# mod_pc = torch.cat((grouped,knn_pc[:,:,:,3:]),dim=3)
# print(mod_pc.shape)
grouped.shape

torch.Size([16, 1024, 32, 7])

In [83]:
x = np.array([[[1,2,3],[4,5,6]],[[7,8,9],[10,11,12]]])
print(x)
print(x.shape)
z = np.array([[0,0],[0,0]])
m = np.array([[0],[0]])
x[z,m,0:1]

[[[ 1  2  3]
  [ 4  5  6]]

 [[ 7  8  9]
  [10 11 12]]]
(2, 2, 3)


array([[[1],
        [1]],

       [[1],
        [1]]])

In [59]:
c = index_points(pc,neighbors.indices)
c.shape

[16, 1024, 32]
[16, 1, 1]
[1, 1024, 32]
torch.Size([1024, 32])


torch.Size([16, 1024, 32, 7])

In [ ]:
for i in range()

In [45]:
c[0][0][:,0:3].shape

torch.Size([32, 3])

In [29]:
print(pc.shape)
print(neighbors.indices.shape)
pc[0,:,:][neighbors.indices[0,:,:]].shape
torch.cat(pc[i,:,:][neighbors.indices[i,:,:]] for i in range(pc.shape[0])).shape

torch.Size([16, 1024, 7])
torch.Size([16, 1024, 32])


TypeError: cat(): argument 'tensors' (position 1) must be tuple of Tensors, not generator

In [39]:
neighbors.indices[-1][0].shape

torch.Size([32])

In [27]:
pu = torch.gather(pc,-1,neighbors.indices.unsqueeze(2))

RuntimeError: Index tensor must have the same number of dimensions as input tensor

In [31]:
torch.index_select(pc,1,neighbors.indices.unsqueeze(-1))

IndexError: index_select(): Index is supposed to be a vector

In [44]:
ind = torch.randn((16,1024,32))
data = torch.randn((16,1024,7))
ind.unsqueeze(2).expand(ind.size(0),ind.size(1),data.size(2)).shape

RuntimeError: expand(torch.FloatTensor{[16, 1024, 1, 32]}, size=[16, 1024, 7]): the number of sizes provided (3) must be greater or equal to the number of dimensions in the tensor (4)

In [14]:
# train_data = TraceParts_Dataloader(split="val")
# train_loader = torch.utils.data.DataLoader(train_data,batch_size=1,shuffle=True)
# print(len(train_loader))
# cuda = torch.device('cuda')
# import time
# import cudf
# from cuml.neighbors import KNeighborsClassifier as cuKNeighbors
# for batch,(pc,label) in enumerate(train_loader):
#     print(batch)
#     s_t = time.time()
#     pc = torch.squeeze(pc,dim=0).numpy()
# #     pc = cudf.DataFrame(pc)
# #     label = cudf.DataFrame(label.numpy())
# #     print(label.shape)
#     label = label.numpy()
#     model = cuKNeighbors(n_neighbors=32)
#     model.fit(pc,label)
#     knn = model.predict(pc)
#     print("Time Taken :",time.time()-s_t)
#     print(knn)
    
    
# #     print("B : {}, pc shape : {}, label shape : {}".format(batch,pc.shape,label.shape))